In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
import matplotlib.pyplot as plt 
import timeit 

#creating sin wave for input values 
T = 20
L = 1000
N = 100 

x = np.empty((N, L), np.float32 )
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1) 
y = np.sin(x / 1.0 / T).astype(np.float32)

#torch.save(data, open('traindata.pt', 'wb'))
#creating and LSTM Predictor
class Sequence(nn.Module):
    def __init__(self,n_hidden = 51):
        super(Sequence, self).__init__()
        self.n_hidden = n_hidden
        #lstm cell1 & cells2, linear layer for prediction 
        self.lstm1 = nn.LSTMCell(1, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden ) 
        self.linear = nn.Linear(self.n_hidden, 1) 
        
    def forward(self, x, future = 0): 
        outputs = []
        n_samples = x.size(0)
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        for input_t in x.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t)) 
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) 
            output = self.linear(h_t2) 
            outputs.append(output) 
        # if we should predict the future 
        for i in range(future): 
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2) 
            outputs.append(output)
        outputs = torch.cat(outputs, dim=1) 
        return outputs 


    #load data and make training set
    # y= 100 , 1000
train_input = torch.from_numpy(y[3:, :-1]) #97, 999 
train_target = torch.from_numpy(y[3:, 1:]) #97, 999
test_input = torch.from_numpy(y[:3, :-1]) # 3, 999
test_target = torch.from_numpy(y[:3, 1:]) # 3, 999 
    
# build the model
model = Sequence() 
criterion = nn.MSELoss() 
    
# use LBFGS as optimizer since we can load the whole data to train 
optimizer = optim.LBFGS(model.parameters(), lr=0.8) 
start = timeit.default_timer() 
    
#begin to train 
n_steps = 10
for i in range(n_steps):
    print('Step', i) #change 
    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target) 
        print('loss:', loss.item()) 
        loss.backward() #back propagation 
        return loss 
    optimizer.step(closure)
            
# begin to predict, no need to track gradient here 
    with torch.no_grad():
        future = 1000 #predict steps can increase 
        pred = model(test_input, future=future) 
        loss = criterion(pred[:,:-future], test_target) 
        print('test loss:', loss.item())
        y = pred.detach().numpy() # numerical outputs 
        
    # draw the result 
    plt.figure(figsize=(12,8)) 
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)' f"\nstep{i+1}", fontsize=20) 
    plt.xlabel('x', fontsize=15)
    plt.ylabel('y', fontsize=15) 
    plt.xticks(fontsize=15) 
    plt.yticks(fontsize=15) 
    n = train_input.shape[1] #999 
    def draw(yi, color):
        plt.plot(np.arange(n), yi[:n], color, linewidth = 2.0) 
        plt.plot(np.arange(n, n+future), yi[n:], color + ':', linewidth = 2.0) 
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b') 
    plt.savefig('predict%d.pdf'%i) 
    plt.close() 


Step 0
loss: 0.5117824673652649
loss: 0.502273440361023
loss: 0.48187577724456787
loss: 0.4698578119277954
loss: 0.4158863425254822
loss: 23.42149543762207
loss: 0.31355908513069153
loss: 0.20930740237236023
loss: 0.03968942165374756
loss: 0.02204533852636814
loss: 0.021877802908420563
loss: 0.021036725491285324
loss: 0.019406547769904137
loss: 0.01629868522286415
loss: 0.011432593688368797
loss: 0.015418821945786476
loss: 0.005477991886436939
loss: 0.004696030635386705
loss: 0.002819222630932927
loss: 0.001863868092186749
test loss: 0.0012473760871216655
Step 1
loss: 0.0012726872228085995
loss: 0.0010519188363105059
loss: 0.0008675462449900806
loss: 0.0007738539716228843
loss: 0.0006875462713651359
loss: 0.0005840663216076791
loss: 0.000494161038659513
loss: 0.00047164628631435335
loss: 0.00046624866081401706
loss: 0.0004633176722563803
loss: 0.00045721096103079617
loss: 0.0004441509081516415
loss: 0.000416094291722402
loss: 0.0003601006173994392
loss: 0.00026444756076671183
loss: 0.0